# Swapped Food Delivery  
**Zomato SQL Interview Question**

---

### Question  
Zomato is a leading online food delivery service that connects users with various restaurants and cuisines, allowing them to browse menus, place orders, and get meals delivered to their doorsteps.

Recently, Zomato encountered an issue with their delivery system. Due to an error in the delivery driver instructions, each item's order was swapped with the item in the subsequent row. As a data analyst, you're asked to correct this swapping error and return the proper pairing of order ID and item.

If the last item has an odd order ID, it should remain as the last item in the corrected data. For example, if the last item is Order ID 7 Tandoori Chicken, then it should remain as Order ID 7 in the corrected data.

In the results, return the correct pairs of order IDs and items.

---

### Schema

**orders Table**  
| column_name | type    | description                         |
|-------------|---------|-------------------------------------|
| order_id    | integer | The ID of each Zomato order.        |
| item        | string  | The name of the food item in order. |

---

### Example Input

Initial (incorrect) data:

| order_id | item             |
|----------|------------------|
| 1        | Chow Mein        |
| 2        | Pizza            |
| 3        | Pad Thai         |
| 4        | Butter Chicken   |
| 5        | Eggrolls         |
| 6        | Burger           |
| 7        | Tandoori Chicken |

---

### Example Output

Corrected data:

| corrected_order_id | item             |
|--------------------|------------------|
| 1                  | Pizza            |
| 2                  | Chow Mein        |
| 3                  | Butter Chicken   |
| 4                  | Pad Thai         |
| 5                  | Burger           |
| 6                  | Eggrolls         |
| 7                  | Tandoori Chicken |

---

### Explanation  
- Order ID 1 is now associated with Pizza  
- Order ID 2 is paired with Chow Mein  
- This pattern continues with each adjacent pair swapped  
- Order ID 7 remains unchanged because it is the last odd order ID in the list  
- This adjustment ensures that each order is correctly aligned with its respective item, resolving the swap error.


In [1]:

from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
from pyspark.sql.functions import *

# Create SparkSession
spark = SparkSession.builder.master('local[1]').appName("SwappedFoodDelivery").getOrCreate()

# Define schema
schema = StructType([
    StructField("order_id", IntegerType(), True),
    StructField("item", StringType(), True)
])

# Sample data as per the example input
data = [
    (1, "Chow Mein"),
    (2, "Pizza"),
    (3, "Pad Thai"),
    (4, "Butter Chicken"),
    (5, "Eggrolls"),
    (6, "Burger"),
    (7, "Tandoori Chicken")
]

# Create DataFrame
orders_df = spark.createDataFrame(data, schema)

# Show the DataFrame
orders_df.show()


+--------+----------------+
|order_id|            item|
+--------+----------------+
|       1|       Chow Mein|
|       2|           Pizza|
|       3|        Pad Thai|
|       4|  Butter Chicken|
|       5|        Eggrolls|
|       6|          Burger|
|       7|Tandoori Chicken|
+--------+----------------+



In [2]:
mx=orders_df.count()
print(mx)
orders_df.withColumn('corrected_order_id',when(((col('order_id')%2!=0) & (col('order_id')!=mx)), col('order_id')+1)
                                         .when(((col('order_id')%2!=0) & (col('order_id')==mx)), col('order_id'))
                                         .otherwise(col('order_id')-1))\
                                         .select('corrected_order_id','item')\
                                         .orderBy('corrected_order_id').show()
                                         

7
+------------------+----------------+
|corrected_order_id|            item|
+------------------+----------------+
|                 1|           Pizza|
|                 2|       Chow Mein|
|                 3|  Butter Chicken|
|                 4|        Pad Thai|
|                 5|          Burger|
|                 6|        Eggrolls|
|                 7|Tandoori Chicken|
+------------------+----------------+



In [3]:
orders_df.createOrReplaceTempView('orders')

spark.sql(
'''
SELECT 
    CASE 
    WHEN order_id%2!=0 and mx!=order_id then order_id+1 
    WHEN order_id%2!=0 and mx=order_id then order_id
    ELSE order_id-1 end
    AS corrected_order_id,
    item
FROM orders cross join (select max(order_id) as mx from orders)
order by 1

'''
).show()

+------------------+----------------+
|corrected_order_id|            item|
+------------------+----------------+
|                 1|           Pizza|
|                 2|       Chow Mein|
|                 3|  Butter Chicken|
|                 4|        Pad Thai|
|                 5|          Burger|
|                 6|        Eggrolls|
|                 7|Tandoori Chicken|
+------------------+----------------+

